# Primitives

The existing Qiskit interface to backends (backend.run()) was originally designed to accept a list of circuits and return counts for every job. Over time, it became clear that users have diverse purposes for quantum computing, and therefore the ways in which they define the requirements for their computing jobs are expanding. 

To address this issue, Qiskit introduced the `qiskit.primitives` module in version 0.35.0 (Qiskit Terra 0.20.0) to provide a simplified way for end users to compute outputs of interest from a QuantumCircuit and to use backends. Qiskit's primitives provide methods that make it easier to build modular algorithms and other higher-order programs. Rather than simply returning counts, they return more immediately meaningful information.

The two main submodules of the primitives are the `Sampler` and the `Estimator`. The first one computes quasi-probability distributions from circuit measurements, while the second one calculates and interprets expectation values of quantum operators that are required for many near-term quantum algorithms.

MQT now provides its own version of the Qiskit primitives:

- `Sampler` leverages the default circuit simulator based on decision diagrams, while preserving the methods and functionality of the original Qiskit's sampler.

- `Estimator` is currently in development and will be available soon.


## Sampler

The `Sampler` takes a list of `QuantumCircuit` objects and simulates them using the `QasmSimulatorBackend` from MQT DDSIM. It then computes the quasi-probability distributions of the circuits in the list and encapsulates the results, along with the job's metadata, within a `SamplerResult` object. 

Furthermore, it also handles transpilation and parameter binding when working with parametrized circuits.

Here we show an example on how to use this submodule:

In [ ]:
import numpy as np
from qiskit import *
from qiskit.circuit import Parameter

from mqt.ddsim.sampler import Sampler

# Parametrized circuit to create an X-gate
theta_a = Parameter("theta_a")
circ = QuantumCircuit(1)
circ.ry(theta_a, 0)
circ.measure_all()

# Show circuit
circ.draw("mpl")

In [ ]:
# Initialize sampler
sampler = Sampler()

The first argument in the `run` method is the circuit to simulate. The second argument contains the parameter values, if needed. The number of shots can be set as an additional argument. If not, the default value is 1024.

In [ ]:
job = sampler.run(circ, np.pi, shots=int(1e4))
result = job.result()

The `result()` method of the job returns a `SamplerResult` object, which includes both the quasi-probability distribution and job metadata.

In [ ]:
print(f">>> {result}")
print(f"  > Quasi-probability distribution: {result.quasi_dists[0]}")

It is also possible to simulate multiple circuits in one job:

In [ ]:
# Create a second circuit
circ_2 = QuantumCircuit(1)
circ_2.measure_all()
circ_2.draw("mpl")

In this case, both the circuits and the parameter values must be entered as a `Sequence`.

In [ ]:
job = sampler.run((circ, circ_2), [[np.pi], []], shots=int(1e4))
result = job.result()

print(f">>> {result}")
print(f"  > Quasi-probability distribution for circuit 1: {result.quasi_dists[0]}")
print(f"  > Quasi-probability distribution for circuit 2: {result.quasi_dists[1]}")

Now let's use the `Sampler` on a more complex circuit. For this example, we will test Grover's algorithm on a 4 qubit circuit where the marked states in the binary representation are "0110" and "1010". 

A similar implementation of Grover's algorithm using Qiskit's sampler can be found here: https://qiskit.org/ecosystem/ibm-runtime/tutorials/grover_with_sampler.html

In [ ]:
# Built-in modules
import math

# Imports from Qiskit
from qiskit.circuit.library import MCMT, GroverOperator, ZGate
from qiskit.visualization import plot_distribution


# Define an oracle circuit that marks the input states, which are represented as a list of bitstrings
def grover_oracle(marked_states):
    if not isinstance(marked_states, list):
        marked_states = [marked_states]
    # Compute the number of qubits in circuit
    num_qubits = len(marked_states[0])

    qc = QuantumCircuit(num_qubits)
    # Mark each target state in the input list
    for target in marked_states:
        # Flip target bit-string to match Qiskit bit-ordering
        rev_target = target[::-1]
        # Find the indices of all the '0' elements in bit-string
        zero_inds = [ind for ind in range(num_qubits) if rev_target.startswith("0", ind)]
        # Add a multi-controlled Z-gate with pre- and post-applied X-gates (open-controls)
        # where the target bit-string has a '0' entry
        qc.x(zero_inds)
        qc.compose(MCMT(ZGate(), num_qubits - 1, 1), inplace=True)
        qc.x(zero_inds)
    return qc


# Enter the states to be marked
marked_states = ["0110", "1010"]
oracle = grover_oracle(marked_states)
oracle.draw("mpl")

In [ ]:
# Build circuit that amplifies the marked states by the oracle

grover_op = GroverOperator(oracle)
grover_op.decompose().draw("mpl")

In [ ]:
# Compute the optimal number of repetitions of the GroverOperator to amplify the probabilities of the marked states
optimal_num_iterations = math.floor(math.pi / 4 * math.sqrt(2**grover_op.num_qubits / len(marked_states)))


# Build the full Grover circuit
qc = QuantumCircuit(grover_op.num_qubits)
# Create even superposition of all basis states
qc.h(range(grover_op.num_qubits))
# Apply Grover operator the optimal number of times
qc.compose(grover_op.power(optimal_num_iterations), inplace=True)
# Measure all qubits
qc.measure_all()
qc.draw("mpl")

After having built the circuit, use the `Sampler` to get the probability distribution.

In [ ]:
dist = sampler.run(qc, shots=int(1e4)).result().quasi_dists[0]
plot_distribution(dist.binary_probabilities())